# Setting up a SQLite server using Python

In [1]:
import sqlite3
from sqlite3 import Error
import os

# GET THE CURRENT WORKING DIRECTORY
path = os.getcwd()

# DEFINE FUNCTIONS TO MAKE THE CONNECTION
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


def create_project(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO projects(name,begin_date,end_date)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid


def create_task(conn, task):
    """
    Create a new task
    :param conn:
    :param task:
    :return:
    """

    sql = ''' INSERT INTO tasks(name,priority,status_id,project_id,begin_date,end_date)
              VALUES(?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid


In [2]:
sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS projects (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    begin_date text,
                                    end_date text
                                ); """

sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                id integer PRIMARY KEY,
                                name text NOT NULL,
                                priority integer,
                                status_id integer NOT NULL,
                                project_id integer NOT NULL,
                                begin_date text NOT NULL,
                                end_date text NOT NULL,
                                FOREIGN KEY (project_id) REFERENCES projects (id)
                            );"""

In [3]:
# create a database connection
conn = create_connection(path + "/pythonsqlite.db")

# create tables
if conn is not None:
    # create projects table
    create_table(conn, sql_create_projects_table)

    # create tasks table
    create_table(conn, sql_create_tasks_table)

    # create a new project
    project = ('Cool App with SQLite & Python', '2015-01-01', '2015-01-30');
    project_id = create_project(conn, project)

    # tasks
    task_1 = ('Analyze the requirements of the app', 1, 1, project_id, '2015-01-01', '2015-01-02')
    task_2 = ('Confirm with user about the top requirements', 1, 1, project_id, '2015-01-03', '2015-01-05')
    task_3 = ('Confirm with user about the top requirements', 2, 2, project_id, '2015-01-03', '2015-01-05')

    # create tasks
    create_task(conn, task_1)
    create_task(conn, task_2)
    create_task(conn, task_3)

else:
    print("Error! cannot create the database connection.")

In [4]:
cur = conn.cursor()
cur.execute("SELECT * FROM projects").fetchall()

[(1, 'Cool App with SQLite & Python', '2015-01-01', '2015-01-30'),
 (2, 'Cool App with SQLite & Python', '2015-01-01', '2015-01-30')]

In [5]:
cur.execute("SELECT * FROM tasks").fetchone()

(2,
 3,
 'Confirm with user about the top requirements',
 2,
 2,
 1,
 '2015-01-03',
 '2015-01-05')

In [6]:
cur.close()

In [7]:
import pandas as pd

# Read sqlite query results into a pandas DataFrame
tasks_df = pd.read_sql_query("SELECT * from tasks", conn)

# Verify that result of SQL query is stored in the dataframe
tasks_df.head()


,index,id,name,priority,status_id,project_id,begin_date,end_date
0,2.0,3.0,Confirm with user about the top requirements,2,2,1,2015-01-03,2015-01-05
1,NaN,NaN,Analyze the requirements of the app,1,1,2,2015-01-01,2015-01-02
2,NaN,NaN,Confirm with user about the top requirements,1,1,2,2015-01-03,2015-01-05
3,NaN,NaN,Confirm with user about the top requirements,2,2,2,2015-01-03,2015-01-05


In [8]:
# Select only data for status_id = 2
tasks_df = tasks_df[tasks_df.status_id == 2]

# Write the new DataFrame to a new SQLite table
tasks_df.to_sql("tasks", conn, if_exists="replace")


2

In [9]:
new_tasks_df = pd.read_sql_query("SELECT * from tasks", conn)
new_tasks_df

,level_0,index,id,name,priority,status_id,project_id,begin_date,end_date
0,0,2.0,3.0,Confirm with user about the top requirements,2,2,1,2015-01-03,2015-01-05
1,3,NaN,NaN,Confirm with user about the top requirements,2,2,2,2015-01-03,2015-01-05
